# Compare BetaVAE models
Compare BetaVAE models w.r.t. the disentanglement scores on HMDB dataset.

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from specvae import utils

In [ ]:
def load_experiment_from_path(filepath):
    return pd.read_csv(filepath, index_col=0)

def load_experiment(dataset, experiment_name, filename='experiment.csv', base_path=None):
    if base_path is None:
        filepath = utils.get_project_path() / '.model' / dataset / experiment_name / filename
    else:
        filepath = base_path / dataset / experiment_name / filename
    return load_experiment_from_path(filepath)

def load_experiment_sessions(dataset, experiment_name, filenames=['experiment.csv'], base_path=None):
    dfs = [load_experiment(dataset, experiment_name, filename, base_path) for filename in filenames]
    merged_df = pd.concat(dfs, ignore_index=True)
    return merged_df

In [ ]:
indices = [0,1]
df = load_experiment_sessions('HMDB', 'betavae_capacity_nextron', 
    ['experiment01_dms.csv', 'experiment02_dms.csv', 'experiment03_dms.csv', 
     'experiment04_dms.csv', 'experiment05_dms.csv', 'experiment06_dms.csv'])
# df2 = load_experiment_sessions('HMDB', 'betavae_capacity_nextron', 
#     ['experiment01.csv', 'experiment02.csv', 'experiment03.csv', 'experiment04.csv', 
#      'experiment05.csv', 'experiment06.csv'])
# df = pd.concat([df1, df2], ignore_index=True)
df.to_csv(utils.get_project_path() / '.model' / 'betavae_hmdb.csv')
df

In [ ]:
columns = list(df.columns)
params = list(filter(lambda x: x.startswith('param_'), columns))
values = list(filter(lambda x: x.startswith('m_'), columns))
others = list(filter(lambda x: not x.startswith('m_') and not x.startswith('param_'), columns))
# Separate by split:
train_values = list(filter(lambda x: x.startswith('m_train_'), columns))
valid_values = list(filter(lambda x: x.startswith('m_valid_'), columns))
test_values = list(filter(lambda x: x.startswith('m_test_'), columns))

In [ ]:
import ast
def is_symmetric(row):
    layer_config = ast.literal_eval(row['layer_config'])
    return len(layer_config[0]) == len(layer_config[1])

def depth(row):
    layer_config = ast.literal_eval(row['layer_config'])
    lencoder, ldecoder = len(layer_config[0]) - 2, len(layer_config[1]) - 2
    return max(lencoder, ldecoder)

df['is_symmetric'] = df.apply(is_symmetric, axis=1)
df['depth'] = df.apply(depth, axis=1)


### Add random disentanglement scores for testing

In [ ]:
# import itertools as it
# pcols =  ['m.train.factor_vae.'       + str(p) for p in it.permutations(indices, len(indices))]
# pcols += ['m.valid.beta_vae.'       + str(p) for p in it.permutations(indices, len(indices))]
# pcols += ['m.train.factor_vae.'     + str(p) for p in it.permutations(indices, len(indices))]
# pcols += ['m.valid.factor_vae.'     + str(p) for p in it.permutations(indices, len(indices))]
# pcols += ['m.train.mig.'            + str(p) for p in it.permutations(indices, len(indices))]
# pcols += ['m.valid.mig.'            + str(p) for p in it.permutations(indices, len(indices))]

# def random_score(row):
#     for n in pcols:
#         row[n] = np.random.random()
#     return row
# df = df.apply(random_score, axis=1)

### Prepare scores

In [ ]:
import itertools as it
def prepare_scores(df, indices=[0,1,2]):
    def unpivot_by_name(df, score_name):
        def split(row):
            if 'train' in row['permutation']:
                row['permutation'] = row['permutation'].replace('m.train.' + score_name + '.', '')
                row['split'] = 'train'
            elif 'eval' in row['permutation']:
                row['permutation'] = row['permutation'].replace('m.eval.' + score_name + '.', '')
                row['split'] = 'valid'
            return row
        vars = ['m.train.' + score_name + '.' + str(p) for p in it.permutations(indices, len(indices))] + \
                ['m.eval.' + score_name + '.' + str(p) for p in it.permutations(indices, len(indices))]
        df1 = df.melt(id_vars=['full_model_name'], value_vars=vars, var_name='permutation', value_name=score_name)
        df1 = df1.apply(split, axis=1)
        return df1
    df1 = unpivot_by_name(df, 'beta_vae')
    df2 = unpivot_by_name(df, 'factor_vae')
    df3 = unpivot_by_name(df, 'mig')
    df_ = pd.merge(df1, df2, on=['full_model_name', 'split', 'permutation'])
    df_ = pd.merge(df_, df3, on=['full_model_name', 'split', 'permutation'])
    df_ = pd.merge(df_, df, on=['full_model_name'])
    return df_


In [ ]:
df1 = prepare_scores(df, indices)
df1

## Top models

In [ ]:
pd.set_option("display.max_colwidth", 200)
pd.set_option('display.max_rows', 200)

### Best models w.r.t. beta_vae score grouped by beta, n_peaks and permutation

In [ ]:
df1[['param_beta', 'param_max_num_peaks', 'permutation', 'layer_config', 'full_model_name', 
    'beta_vae', 'factor_vae', 'mig',
    'm_train_cos_sim', 'm_train_eu_dist', 'm_train_per_diff']].loc[
    df1.groupby(['param_beta', 'param_max_num_peaks', 'permutation'])['beta_vae'].idxmax()]

### Best models w.r.t. factor_vae score grouped by beta, n_peaks and permutation

In [ ]:
df1[['param_beta', 'param_max_num_peaks', 'permutation', 'layer_config', 'full_model_name', 
    'beta_vae', 'factor_vae', 'mig',
    'm_train_cos_sim', 'm_train_eu_dist', 'm_train_per_diff']].loc[
    df1.groupby(['param_beta', 'param_max_num_peaks', 'permutation'])['factor_vae'].idxmax()]

### Best models w.r.t. MIG score grouped by beta, n_peaks and permutation

In [ ]:
df1[['param_beta', 'param_max_num_peaks', 'permutation', 'layer_config', 'full_model_name', 
    'beta_vae', 'factor_vae', 'mig',
    'm_train_cos_sim', 'm_train_eu_dist', 'm_train_per_diff']].loc[
    df1.groupby(['param_beta', 'param_max_num_peaks', 'permutation'])['mig'].idxmax()]

### Scores for each permutation

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(60, 20))
sns.boxplot(data=df1, x='param_beta', y='beta_vae', hue='permutation', palette='viridis', ax=axs[0])
sns.boxplot(data=df1, x='param_beta', y='factor_vae', hue='permutation', palette='viridis', ax=axs[1])
sns.boxplot(data=df1, x='param_beta', y='mig', hue='permutation', palette='viridis', ax=axs[2])
axs[0].legend([])
axs[1].legend([])
plt.legend(loc='upper right')

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(30, 20))
axs_ = axs.ravel()
m = ['BetaVAE', 'FactorVAE', 'MIG']
for i, perm in enumerate(it.permutations(indices, len(indices))):
    for k in range(3):
        axs_[3*i+k].set_title('Metric=' + m[k] + ' | ' + 'permutation='+str(perm)) 
    sns.violinplot(data=df1[df1['permutation'] == str(perm)], x='param_beta', y='beta_vae', hue='split', ax=axs_[3*i])
    sns.violinplot(data=df1[df1['permutation'] == str(perm)], x='param_beta', y='factor_vae', hue='split', ax=axs_[3*i+1])
    sns.violinplot(data=df1[df1['permutation'] == str(perm)], x='param_beta', y='mig', hue='split', ax=axs_[3*i+2])

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(60, 20))
axs_ = axs.ravel()
m = ['BetaVAE', 'FactorVAE', 'MIG']
for i, perm in enumerate(it.permutations(indices, len(indices))):
    for k in range(3):
        axs_[3*i+k].set_title('Metric=' + m[k] + ' | ' + 'permutation='+str(perm)) 
    sns.violinplot(data=df1[df1['permutation'] == str(perm)], x='param_beta', y='beta_vae', hue='param_max_num_peaks', ax=axs_[3*i])
    sns.violinplot(data=df1[df1['permutation'] == str(perm)], x='param_beta', y='factor_vae', hue='param_max_num_peaks', ax=axs_[3*i+1])
    sns.violinplot(data=df1[df1['permutation'] == str(perm)], x='param_beta', y='mig', hue='param_max_num_peaks', ax=axs_[3*i+2])

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(30, 20))
axs_ = axs.ravel()
m = ['BetaVAE', 'FactorVAE', 'MIG']
for i, perm in enumerate(it.permutations(indices, len(indices))):
    for k in range(3):
        axs_[3*i+k].set_title('Metric=' + m[k] + ' | ' + 'permutation='+str(perm)) 
    sns.stripplot(data=df1[df1['permutation'] == str(perm)], x='param_beta', y='beta_vae', hue='param_max_num_peaks', ax=axs_[3*i])
    sns.stripplot(data=df1[df1['permutation'] == str(perm)], x='param_beta', y='factor_vae', hue='param_max_num_peaks', ax=axs_[3*i+1])
    sns.stripplot(data=df1[df1['permutation'] == str(perm)], x='param_beta', y='mig', hue='param_max_num_peaks', ax=axs_[3*i+2])

### Disentanglement scores rank correlation per permutation

In [ ]:
import scipy.stats
cmap = sns.diverging_palette(230, 20, as_cmap=True)

for perm in it.permutations(indices, len(indices)):
    dfp_train = df1[df1['permutation'].isin([str(perm)]) & df1['split'].isin(['train'])][['beta_vae', 'factor_vae', 'mig']]
    dfp_valid = df1[df1['permutation'].isin([str(perm)]) & df1['split'].isin(['valid'])][['beta_vae', 'factor_vae', 'mig']]
    c1, p1 = scipy.stats.spearmanr(dfp_train)
    c2, p2 = scipy.stats.spearmanr(dfp_valid)
    
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].set_title('Train, permutation: ' + str(perm))
    axs[1].set_title('Valid, permutation: ' + str(perm))
    sns.heatmap(c1, cmap=cmap, square=True, ax=axs[0], vmin=0.0, vmax=1.0, annot=True, linewidths=.5, center=0.5,
        xticklabels=['beta_vae', 'factor_vae', 'mig'], yticklabels=['beta_vae', 'factor_vae', 'mig'], cbar=False)
    sns.heatmap(c2, cmap=cmap, square=True, ax=axs[1], vmin=0.0, vmax=1.0, annot=True, linewidths=.5, center=0.5,
        xticklabels=['beta_vae', 'factor_vae', 'mig'], yticklabels=['beta_vae', 'factor_vae', 'mig'], cbar=False)
    fig.colorbar(axs[0].get_children()[0], ax=axs.ravel().tolist(), aspect=10., anchor=(1.6, 0.5), shrink=0.97)
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()


### Pairwise scatter plot for different disentanglement metrics

In [ ]:
for n_peaks in df1['param_max_num_peaks'].unique():
    g = sns.pairplot(df1[df1['param_max_num_peaks'] == n_peaks][['beta_vae', 'factor_vae', 'mig', 'param_beta']], hue='param_beta', height=4)
    g.fig.suptitle('n_peaks=' + str(n_peaks))

In [ ]:
for beta in df1['param_beta'].unique():
    g = sns.pairplot(df1[df1['param_beta'] == beta][['beta_vae', 'factor_vae', 'mig', 'param_max_num_peaks']], hue='param_max_num_peaks', height=4)
    g.fig.suptitle('beta=' + str(beta))

### Variance of the disentanglement scores explained by different factors

#### Preprocess factors

In [ ]:
# Assign categorical id to continuous variable: param_beta
param_beta_unique = df1['param_beta'].unique()
param_beta_map = dict(zip(param_beta_unique, range(len(param_beta_unique))))
df1['param_beta_id'] = df1.apply(lambda row: param_beta_map[row['param_beta']], axis=1)
param_beta_map, df1['param_beta_id'].unique()

In [ ]:
# Assign categorical id to continuous variable: n_peaks
n_peaks_unique = df1['param_max_num_peaks'].unique()
n_peaks_map = dict(zip(n_peaks_unique, range(len(n_peaks_unique))))
df1['param_max_num_peaks_id'] = df1.apply(lambda row: n_peaks_map[row['param_max_num_peaks']], axis=1)
n_peaks_map, df1['param_max_num_peaks_id'].unique()

In [ ]:
# Assign categorical id to continuous variable: layer_config
def reduce_layer_config(row):
    layer_config = ast.literal_eval(row['layer_config'])
    encoder, decoder = layer_config[0][1:], layer_config[1][:-1]
    return str([encoder, decoder])

df1['arch'] = df1.apply(reduce_layer_config, axis=1)
arch_unique = df1['arch'].unique()
arch_map = dict(zip(arch_unique, range(len(arch_unique))))
df1['arch_id'] = df1.apply(lambda row: arch_map[row['arch']], axis=1)
arch_map, df1['arch_id'].unique()

#### One hot encode

In [ ]:
def one_hot_encode(df, name):
    id = df[name].to_numpy()
    ids = np.unique(id)
    n_values = np.max(ids) + 1
    return np.eye(n_values)[id]

In [ ]:
param_beta_ohe = one_hot_encode(df1, 'param_beta_id')
n_peaks_ohe = one_hot_encode(df1, 'param_max_num_peaks_id')
arch_ohe = one_hot_encode(df1, 'arch_id')
param_beta_ohe.shape, n_peaks_ohe.shape, arch_ohe.shape

In [ ]:
from sklearn.linear_model import LinearRegression
import sklearn.metrics as skm

def linear_regression(X, y):
    reg = LinearRegression().fit(X, y)
    y_pred = reg.predict(X)

    mse = skm.mean_squared_error(y, y_pred)
    mae = skm.mean_absolute_error(y, y_pred)
    me  = skm.max_error(y, y_pred)
    evs = skm.explained_variance_score(y, y_pred)
    return mse, mae, me, evs

In [ ]:
scores = []

In [ ]:
score_name = 'beta_vae'
X = np.hstack((param_beta_ohe, n_peaks_ohe, arch_ohe))
y = df1[score_name].to_numpy()
mse, mae, me, evs = linear_regression(X, y)
scores.append([mse, mae, me, evs, score_name, 1, 1, 1])

X = np.hstack((param_beta_ohe, n_peaks_ohe))
y = df1[score_name].to_numpy()
mse, mae, me, evs = linear_regression(X, y)
scores.append([mse, mae, me, evs, score_name, 1, 1, 0])

X = np.hstack((param_beta_ohe, arch_ohe))
y = df1[score_name].to_numpy()
mse, mae, me, evs = linear_regression(X, y)
scores.append([mse, mae, me, evs, score_name, 1, 0, 1])

X = np.hstack((n_peaks_ohe, arch_ohe))
y = df1[score_name].to_numpy()
mse, mae, me, evs = linear_regression(X, y)
scores.append([mse, mae, me, evs, score_name, 0, 1, 1])

X = param_beta_ohe
y = df1[score_name].to_numpy()
mse, mae, me, evs = linear_regression(X, y)
scores.append([mse, mae, me, evs, score_name, 1, 0, 0])

X = n_peaks_ohe
y = df1[score_name].to_numpy()
mse, mae, me, evs = linear_regression(X, y)
scores.append([mse, mae, me, evs, score_name, 0, 1, 0])

X = arch_ohe
y = df1[score_name].to_numpy()
mse, mae, me, evs = linear_regression(X, y)
scores.append([mse, mae, me, evs, score_name, 0, 0, 1])

In [ ]:
score_name = 'factor_vae'
X = np.hstack((param_beta_ohe, n_peaks_ohe, arch_ohe))
y = df1[score_name].to_numpy()
mse, mae, me, evs = linear_regression(X, y)
scores.append([mse, mae, me, evs, score_name, 1, 1, 1])

X = np.hstack((param_beta_ohe, n_peaks_ohe))
y = df1[score_name].to_numpy()
mse, mae, me, evs = linear_regression(X, y)
scores.append([mse, mae, me, evs, score_name, 1, 1, 0])

X = np.hstack((param_beta_ohe, arch_ohe))
y = df1[score_name].to_numpy()
mse, mae, me, evs = linear_regression(X, y)
scores.append([mse, mae, me, evs, score_name, 1, 0, 1])

X = np.hstack((n_peaks_ohe, arch_ohe))
y = df1[score_name].to_numpy()
mse, mae, me, evs = linear_regression(X, y)
scores.append([mse, mae, me, evs, score_name, 0, 1, 1])

X = param_beta_ohe
y = df1[score_name].to_numpy()
mse, mae, me, evs = linear_regression(X, y)
scores.append([mse, mae, me, evs, score_name, 1, 0, 0])

X = n_peaks_ohe
y = df1[score_name].to_numpy()
mse, mae, me, evs = linear_regression(X, y)
scores.append([mse, mae, me, evs, score_name, 0, 1, 0])

X = arch_ohe
y = df1[score_name].to_numpy()
mse, mae, me, evs = linear_regression(X, y)
scores.append([mse, mae, me, evs, score_name, 0, 0, 1])

In [ ]:
score_name = 'mig'
X = np.hstack((param_beta_ohe, n_peaks_ohe, arch_ohe))
y = df1[score_name].to_numpy()
mse, mae, me, evs = linear_regression(X, y)
scores.append([mse, mae, me, evs, score_name, 1, 1, 1])

X = np.hstack((param_beta_ohe, n_peaks_ohe))
y = df1[score_name].to_numpy()
mse, mae, me, evs = linear_regression(X, y)
scores.append([mse, mae, me, evs, score_name, 1, 1, 0])

X = np.hstack((param_beta_ohe, arch_ohe))
y = df1[score_name].to_numpy()
mse, mae, me, evs = linear_regression(X, y)
scores.append([mse, mae, me, evs, score_name, 1, 0, 1])

X = np.hstack((n_peaks_ohe, arch_ohe))
y = df1[score_name].to_numpy()
mse, mae, me, evs = linear_regression(X, y)
scores.append([mse, mae, me, evs, score_name, 0, 1, 1])

X = param_beta_ohe
y = df1[score_name].to_numpy()
mse, mae, me, evs = linear_regression(X, y)
scores.append([mse, mae, me, evs, score_name, 1, 0, 0])

X = n_peaks_ohe
y = df1[score_name].to_numpy()
mse, mae, me, evs = linear_regression(X, y)
scores.append([mse, mae, me, evs, score_name, 0, 1, 0])

X = arch_ohe
y = df1[score_name].to_numpy()
mse, mae, me, evs = linear_regression(X, y)
scores.append([mse, mae, me, evs, score_name, 0, 0, 1])

In [ ]:
df_scores = pd.DataFrame(scores, columns=['MSE', 'MAE', 'ME', 'EVS', 'target_value', 'param_beta', 'n_peaks', 'arch'])
df_scores

In [ ]:
index = df_scores[df_scores['target_value'] == 'beta_vae'].copy()[['param_beta', 'n_peaks', 'arch']]
beta_vae_score = df_scores[df_scores['target_value'] == 'beta_vae'].copy()[['MSE', 'MAE', 'ME', 'EVS']]
factor_vae_score = df_scores[df_scores['target_value'] == 'factor_vae'].copy()[['MSE', 'MAE', 'ME', 'EVS']]
factor_vae_score.index = beta_vae_score.index
mig_score = df_scores[df_scores['target_value'] == 'mig'].copy()[['MSE', 'MAE', 'ME', 'EVS']]
mig_score.index = factor_vae_score.index
df_scores_ = pd.concat([beta_vae_score, factor_vae_score, mig_score], axis=1, ignore_index=True)
df_scores_= df_scores_.rename(columns={
    0: "beta_vae_MSE", 1: "beta_vae_MAE", 2: 'beta_vae_ME', 3: 'beta_vae_EVS',
    4: "factor_vae_MSE", 5: "factor_vae_MAE", 6: 'factor_vae_ME', 7: 'factor_vae_EVS',
    8: "mig_MSE", 9: "mig_MAE", 10: 'mig_ME', 11: 'mig_EVS'})

# for col in df_scores_.columns:
#     df_scores_[col] = np.log10(df_scores_[col] + 1.)
df_scores = pd.concat([df_scores_, index], axis=1)
df_scores['config'] = 'Config'
df_scores

In [ ]:
# fig, axs = plt.subplots(figsize=(15, 15))
# axs.scatter(y, y_pred)
# axs.plot([np.min(y) - 0.01, np.max(y) + 0.01], [np.min(y_pred) - 0.01, np.max(y_pred) + 0.01], linestyle='dashed', color='red')

In [ ]:
from specvae.visualize import multi_index_heatmap

In [ ]:
fig = multi_index_heatmap(df_scores,
    feature_column_name='config', 
    row_index_columns=['param_beta', 'n_peaks', 'arch'], 
    sample_columns=['beta_vae_MSE', 'factor_vae_MSE', 'mig_MSE'])

In [ ]:
fig = multi_index_heatmap(df_scores,
    feature_column_name='config', 
    row_index_columns=['param_beta', 'n_peaks', 'arch'], 
    sample_columns=['beta_vae_MAE', 'factor_vae_MAE', 'mig_MAE'])

In [ ]:
fig = multi_index_heatmap(df_scores,
    feature_column_name='config', 
    row_index_columns=['param_beta', 'n_peaks', 'arch'], 
    sample_columns=['beta_vae_ME', 'factor_vae_ME', 'mig_ME'])

In [ ]:
fig = multi_index_heatmap(df_scores,
    feature_column_name='config', 
    row_index_columns=['param_beta', 'n_peaks', 'arch'], 
    sample_columns=['beta_vae_EVS', 'factor_vae_EVS', 'mig_EVS'])